<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=eac15875a3a4dce1cd6ac1abfd20207f72ed58bcbbd4a88ffe75b7400bacec46
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-16 10:19:14
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: 53.34 K (0.38%)
Current PnL: -21.53 L (-14.34%)
CY Booked + Current PnL: -9.63 L (-6.41%)
-------------------
Total profit:  1.66 L
Total loss:  -23.19 L
-------------------
Total Booked + Current PnL: 17.35 L (14.09%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.5%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.72 L (63.39%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.78,57.0,X-LC,3.78,214062.0,9562.0,13743.0,0.24,4.68,6.42,11.39,10.0,0.70,1.54,14.05,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,95.94,48.0,M-SC,1.67,84410.0,-16367.0,16460.0,0.14,-16.24,19.50,0.09,245.0,-0.99,0.61,10.90,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.02,57.0,X-MC,1.10,211815.0,20073.0,17687.0,1.47,10.47,8.35,19.69,99.0,1.13,1.53,17.98,XY25,NTT,AC
33,ICICIGI,2260.25,-13.61,65.0,X-MC,2.06,148585.0,12452.0,18677.0,-0.61,9.15,12.57,22.86,91.0,0.67,1.07,22.91,X40,ATH,INSURANCE
56,RELIANCE,1533.00,-14.04,48.0,X-LC,2.85,214781.0,4415.0,24378.0,0.18,2.10,11.35,13.68,37.0,0.18,1.55,18.97,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SONACOMS,1006.0,-33.91,69.0,M-SC,12.27,82241.0,-18918.0,93812.0,3.91,-18.70,114.07,74.03,202.0,-0.20,0.59,15.97,AR,BTT,AUTO
84,WHIRLPOOL,2270.0,-45.59,49.0,M-SC,4.26,92032.0,534.0,78218.0,3.73,0.58,84.99,86.07,223.0,0.01,0.66,33.65,XR,NTT,DURABLES
24,EASEMYTRIP,26.4,-17.82,44.0,M-SC,25.53,79235.0,-97580.0,178374.0,3.44,-55.19,225.12,45.70,197.0,-0.55,0.57,3.44,XY24,NTT,TRAVEL
63,SHALBY,327.0,1185.49,56.0,M-SC,23.00,174775.0,-6368.0,51838.0,3.00,-3.52,29.66,25.10,235.0,-0.12,1.26,40.06,XY24,NTT,HEALTHCARE
4,ANGELONE,3033.0,16.33,70.0,X-SC,9.31,206230.0,15224.0,42483.0,2.85,7.97,20.60,30.21,157.0,0.36,1.49,30.52,X40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HATHWAY,31.40,852.90,36.0,H-SC,5.20,82200.0,-26520.0,106202.0,-1.86,-24.39,129.20,73.29,161.0,-0.25,0.59,9.95,XR,NTT,ENTERTAINMENT
10,BANDHANBNK,400.00,-13.31,46.0,H-SC,2.90,216410.0,-62149.0,319594.0,-1.04,-22.31,147.68,92.42,151.0,-0.19,1.56,25.07,XY24,NTT,BANKS
50,PGHH,17907.65,-26.71,57.0,X-MC,3.25,211980.0,11160.0,56641.0,-1.02,5.56,26.72,33.76,80.0,0.20,1.53,11.04,X40,ATH,FMCG
15,CAMPUS,393.00,-22.96,59.0,M-SC,2.79,160458.0,-7317.0,63156.0,-0.97,-4.36,39.36,33.28,210.0,-0.12,1.16,24.37,XY24,NTT,FOOTWEAR
39,INFY,2275.00,-19.96,45.0,X-LC,7.55,311704.0,-1184.0,172871.0,-0.75,-0.38,55.46,54.87,3.0,-0.01,2.24,6.25,X40,BTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-45.59,49.0,M-SC,4.26,92032.0,534.0,78218.0,3.73,0.58,84.99,86.07,223.0,0.01,0.66,33.65,XR,NTT,DURABLES
60,SAMMAANCAP,326.00,-186.36,68.0,M-SC,18.48,148500.0,-720.0,144906.0,0.01,-0.48,97.58,96.62,208.0,-0.00,1.07,60.71,XY25,NTT,FINANCE
59,SAIL,228.00,42.74,49.0,M-MC,12.20,225410.0,448.0,167209.0,0.47,0.20,74.18,74.53,192.0,0.00,1.62,32.52,XY24,BTT,STEEL
36,INDIAMART,4810.62,-55.57,38.0,H-SC,8.06,124232.0,896.0,130729.0,0.61,0.73,105.23,106.72,119.0,0.01,0.89,24.27,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,107.70,60.0,H-SC,11.90,131220.0,-11259.0,30784.0,0.06,-7.90,23.46,13.70,163.0,-0.37,0.94,54.88,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,95.94,48.0,M-SC,1.67,84410.0,-16367.0,16460.0,0.14,-16.24,19.50,0.09,245.0,-0.99,0.61,10.90,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.79,51.0,H-SC,1.93,221985.0,-47682.0,84021.0,0.40,-17.68,37.85,13.47,138.0,-0.57,1.60,12.87,XY24,NTT,PAINTS
17,CERA,9475.0,-23.39,42.0,H-SC,2.20,139506.0,-36397.0,78416.0,0.21,-20.69,56.21,23.89,149.0,-0.46,1.00,20.48,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.38,45.0,H-MC,7.16,102315.0,-29520.0,71743.0,0.33,-22.39,70.12,32.03,98.0,-0.41,0.74,14.60,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,53.0,M-LC,6.14,207790.0,4620.0,37506.0,0.53,2.27,18.05,20.74,55.0,0.12,1.50,7.94,XY25,NTT,FINANCE
59,SAIL,228.00,42.74,49.0,M-MC,12.20,225410.0,448.0,167209.0,0.47,0.20,74.18,74.53,192.0,0.00,1.62,32.52,XY24,BTT,STEEL
31,HINDZINC,730.22,36.19,67.0,M-LC,9.49,221741.0,16665.0,95903.0,-0.67,8.13,43.25,54.89,52.0,0.17,1.60,34.18,X5K,ATH,METALS
85,WIPRO,420.00,-11.98,54.0,M-LC,5.86,154529.0,3584.0,105868.0,-0.39,2.37,68.51,72.51,53.0,0.03,1.11,8.54,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,54.0,M-LC,8.72,157600.0,16760.0,27738.0,-0.59,11.90,17.60,31.59,66.0,0.60,1.13,41.54,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,36.19,67.0,M-LC,9.49,221741.0,16665.0,95903.0,-0.67,8.13,43.25,54.89,52.0,0.17,1.60,34.18,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,61.0,H-MC,13.02,193404.0,13596.0,98810.0,-0.47,7.56,51.09,62.52,88.0,0.14,1.39,44.23,XR,NTT,BANKS
85,WIPRO,420.00,-11.98,54.0,M-LC,5.86,154529.0,3584.0,105868.0,-0.39,2.37,68.51,72.51,53.0,0.03,1.11,8.54,XR,NTT,IT
84,WHIRLPOOL,2270.00,-45.59,49.0,M-SC,4.26,92032.0,534.0,78218.0,3.73,0.58,84.99,86.07,223.0,0.01,0.66,33.65,XR,NTT,DURABLES
36,INDIAMART,4810.62,-55.57,38.0,H-SC,8.06,124232.0,896.0,130729.0,0.61,0.73,105.23,106.72,119.0,0.01,0.89,24.27,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,5.42,144075.0,-132448.0,242521.0,1.55,-47.90,168.33,39.81,54.0,-0.55,1.04,1.55,XY24,NTT,AUTO
28,HAVELLS,2069.16,-7.88,35.0,X-MC,1.51,228415.0,-19316.0,94381.0,1.04,-7.80,41.32,30.30,92.0,-0.20,1.64,4.46,X40,ATH,ELECTRICAL
51,QUESS,986.00,-47.11,36.0,X-SC,38.51,52341.0,-12665.0,163592.0,0.29,-19.48,312.55,232.18,198.0,-0.08,0.38,2.07,XY24,NTT,MISC
12,BATAINDIA,2096.00,-36.58,40.0,X-SC,4.33,94902.0,-33768.0,79063.0,1.75,-26.24,83.31,35.20,219.0,-0.43,0.68,8.93,X40,NTT,FOOTWEAR
34,ICICIPRULI,790.00,-23.06,40.0,X-MC,2.21,132436.0,-3352.0,46101.0,-0.34,-2.47,34.81,31.48,107.0,-0.07,0.95,9.35,X40,ATH,INSURANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.18,48.0,H-LC,0.66,157300.0,-28795.0,76275.0,0.88,-15.47,48.49,25.51,15.0,-0.38,1.13,15.92,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,1.02,57.0,X-MC,1.10,211815.0,20073.0,17687.0,1.47,10.47,8.35,19.69,99.0,1.13,1.53,17.98,XY25,NTT,AC
28,HAVELLS,2069.16,-7.88,35.0,X-MC,1.51,228415.0,-19316.0,94381.0,1.04,-7.80,41.32,30.30,92.0,-0.20,1.64,4.46,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.61,65.0,X-MC,2.06,148585.0,12452.0,18677.0,-0.61,9.15,12.57,22.86,91.0,0.67,1.07,22.91,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-6.70,63.0,X-LC,2.40,287024.0,21598.0,35189.0,0.43,8.14,12.26,21.40,11.0,0.61,2.07,15.32,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,5.42,144075.0,-132448.0,242521.0,1.55,-47.90,168.33,39.81,54.0,-0.55,1.04,1.55,XY24,NTT,AUTO
51,QUESS,986.00,-47.11,36.0,X-SC,38.51,52341.0,-12665.0,163592.0,0.29,-19.48,312.55,232.18,198.0,-0.08,0.38,2.07,XY24,NTT,MISC
73,TCS,4389.97,-30.29,43.0,X-LC,12.50,277488.0,-68468.0,135164.0,-0.60,-19.79,48.71,19.28,1.0,-0.51,2.00,2.58,X40,ATH,IT
41,ITC,452.00,-39.19,46.0,X-LC,2.69,196211.0,-3927.0,24821.0,0.34,-1.96,12.65,10.44,4.0,-0.16,1.41,3.49,X40,NTT,FMCG
21,COLPAL,3767.14,-4.10,44.0,X-MC,7.90,223790.0,-39575.0,152916.0,0.48,-15.03,68.33,43.04,84.0,-0.26,1.61,3.90,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.11,36.0,X-SC,38.51,52341.0,-12665.0,163592.0,0.29,-19.48,312.55,232.18,198.0,-0.08,0.38,2.07,XY24,NTT,MISC
55,RELAXO,1176.00,-39.20,44.0,X-SC,4.00,82833.0,-62327.0,138257.0,-0.32,-42.94,166.91,52.31,136.0,-0.45,0.60,10.88,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.58,40.0,X-SC,4.33,94902.0,-33768.0,79063.0,1.75,-26.24,83.31,35.20,219.0,-0.43,0.68,8.93,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-4.94,51.0,X-SC,1.00,108088.0,6084.0,37842.0,0.63,5.96,35.01,43.06,122.0,0.16,0.78,24.10,X40N,ATH,MISC
32,HONAUT,58357.33,-23.12,45.0,X-SC,2.40,108345.0,-19593.0,66730.0,-0.11,-15.31,61.59,36.84,143.0,-0.29,0.78,10.92,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-30.29,43.0,X-LC,12.50,277488.0,-68468.0,135164.0,-0.60,-19.79,48.71,19.28,1.0,-0.51,2.00,2.58,X40,ATH,IT
39,INFY,2275.00,-19.96,45.0,X-LC,7.55,311704.0,-1184.0,172871.0,-0.75,-0.38,55.46,54.87,3.0,-0.01,2.24,6.25,X40,BTT,IT
41,ITC,452.00,-39.19,46.0,X-LC,2.69,196211.0,-3927.0,24821.0,0.34,-1.96,12.65,10.44,4.0,-0.16,1.41,3.49,X40,NTT,FMCG
82,VBL,671.64,-19.44,49.0,X-LC,10.64,290258.0,-25584.0,138250.0,2.16,-8.10,47.63,35.67,5.0,-0.19,2.09,4.63,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,51.0,H-LC,7.24,250320.0,-11299.0,130517.0,0.80,-4.32,52.14,45.57,7.0,-0.09,1.80,6.51,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6638.89,53.0,L-SC,6.13,73968.0,-19842.0,96462.0,0.46,-21.15,130.41,81.67,269.0,-0.21,0.53,43.24,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,54.0,L-SC,28.83,83693.0,-29856.0,69909.0,2.30,-26.29,83.53,35.27,268.0,-0.43,0.60,104.84,XR,NTT,HOTELS
48,MASFIN,398.61,-20.51,42.0,H-SC,9.47,90255.0,-7725.0,29324.0,-0.08,-7.88,32.49,22.05,152.0,-0.26,0.65,31.19,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,107.70,60.0,H-SC,11.90,131220.0,-11259.0,30784.0,0.06,-7.90,23.46,13.70,163.0,-0.37,0.94,54.88,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,54.0,H-SC,10.30,145350.0,-33721.0,99448.0,0.30,-18.83,68.42,36.71,135.0,-0.34,1.05,44.22,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,68.0,M-SC,18.48,148500.0,-720.0,144906.0,0.01,-0.48,97.58,96.62,208.0,-0.00,1.07,60.71,XY25,NTT,FINANCE
4,ANGELONE,3033.00,16.33,70.0,X-SC,9.31,206230.0,15224.0,42483.0,2.85,7.97,20.60,30.21,157.0,0.36,1.49,30.52,X40N,NTT,FINANCE
68,SURYODAY,240.00,57.89,54.0,H-SC,10.30,145350.0,-33721.0,99448.0,0.30,-18.83,68.42,36.71,135.0,-0.34,1.05,44.22,XR,NTT,BANKS
63,SHALBY,327.00,1185.49,56.0,M-SC,23.00,174775.0,-6368.0,51838.0,3.00,-3.52,29.66,25.10,235.0,-0.12,1.26,40.06,XY24,NTT,HEALTHCARE
31,HINDZINC,730.22,36.19,67.0,M-LC,9.49,221741.0,16665.0,95903.0,-0.67,8.13,43.25,54.89,52.0,0.17,1.60,34.18,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.52
1,25,43.97
2,50,75.85


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.52
LC    33.69
MC    21.83
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.64
X40      14.72
XY25     12.18
XR       10.98
X40N     10.25
AR        8.22
OX40N     7.66
X200      1.83
X5K       1.60
SR        1.14
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.57
X-LC    22.56
X-MC    16.09
M-SC    12.76
M-LC     5.34
H-LC     4.76
X-SC     4.71
H-MC     3.82
M-MC     1.62
L-SC     1.48
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.10,-4.86,39.42
IT,12.94,-17.15,80.52
FINANCE,12.25,-11.23,61.65
MISC,6.90,-20.09,86.64
BANKS,6.37,-12.61,71.85
PAINTS,5.56,-19.55,37.37
ELECTRICAL,5.35,-10.61,49.42
INSURANCE,3.92,-1.48,39.41
AUTO,2.86,-44.92,114.20


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3320186.0,22
XR,1383256.0,14
AR,1187851.0,9
X40,792931.0,10
X40N,605567.0,8
OX40N,584809.0,10
XY25,494333.0,8
SR,264221.0,2
X5K,95903.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3452790.0,25
M-SC,1271036.0,15
X-LC,1107653.0,13
X-MC,1091637.0,11
X-SC,527967.0,6
H-MC,349797.0,3
H-LC,278718.0,3
M-LC,267015.0,4
L-SC,255566.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1199723.0      6
           XR         824712.0      7
           AR         820205.0      5
M-SC       XY24       687529.0      6
X-MC       XY24       588039.0      4
X-LC       X40        498068.0      5
           XY24       307112.0      2
X-SC       X40N       285312.0      4
H-SC       OX40N      273222.0      4
           SR         264221.0      2
X-MC       X40        215800.0      4
H-LC       AR         206792.0      2
X-LC       X40N       203060.0      3
H-MC       XY24       179244.0      1
X-MC       XY25       170603.0      2
M-MC       XY24       167209.0      1
L-SC       XR         166371.0      2
X-SC       XY24       163592.0      1
M-SC       AR         160854.0      2
           OX40N      150649.0      4
           XY25       144906.0      1
           XR         127098.0      2
X-MC       X40N       117195.0      1
M-LC       XR         105868.0      1
X-LC       XY25        99413.0      3
H-MC       XR          98810.0      1
M-LC       X5K         95903.0      1
L-SC       OX40N       89195.0      1
X-SC       X40         79063.0      1
H-LC       X200        71926.0      1
H-MC       OX40N       71743.0      1
H-SC       MH          70707.0      1
L-MC       XR          60397.0      1
L-LC       XY25        41905.0      1
M-LC       XY25        37506.0      1
           XY24        27738.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
